<a href="https://colab.research.google.com/github/ti-molodykh/colab/blob/main/%D0%9C%D0%BE%D0%BB%D0%BE%D0%B4%D1%8B%D1%85_%22%D0%94%D0%97_2_%D0%92%D0%B8%D1%82%D1%80%D0%B8%D0%BD%D0%B0_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание №2. Атрибуция каналов привлечения

Вы работаете аналитиком в интернет-магазине Ascent: [ссылка](http://shop-stdata.tilda.ws/). Вам пришло сообщение от вашего коллеги-аналитика:

![Условие](https://sun9-46.userapi.com/s/v1/if2/v72TY8B5Dya-tYzTT4IvzEmUi52aYDzOWKwzuPjtp2ScQmmduHo4E4TEVuTsWx3ibZgstaWtQIQTgRIb2v93RnZp.jpg?size=1496x820&quality=95&type=album)

Вашей задачей является реализация данной витрины, результат сохраните в файл order_acquisition.csv. Также вам нужно описать логику присвоения каналов, которую вы посчитали релевантной для данной бизнес-задачи. Весь python-код и sql-код нужно занести в этот jupyter-notebook. 

### Описание бизнес-логики присвоения каналов
Ответьте на вопрос коллег из маркетинга и business development: по какой логике мы устанавливаем источник трафика для юзера? 


In [ ]:
  Так как нет, четкого соответствия между таблицей user_orders и user_x_channel (хотя добавление поля даты в таблицу user_x_channel нам бы позволило с помощью
диапозонного разбиения дать это соответствие, можно заложиться на дату платежа и дату заказа но не каждый заказ может быть оплачен и разница в датах может быть
колосальной. Считаю, что отсутствие четкого соответствия между таблицей заказов и таблицей каналов прихода это не полная проработка архитектуры приложения,
которую необходимо доработать), то можно судить только по косвенным признакам - а именно разбивать клиентов на категории и выбирать
наиболее релевантных лидов (тех клиентов которые стали нашими покупателями). 
  Наиболее релевантные клиенты это те кто кроме визита, создал заказ и оплатил (да у нас нету, четкого соответствия между заказ-оплата и как можно сделать из 
этого вывод между заказ-оплата ведется просто баланс (заказ -, оплата +)и во время передачи заказа баланс должен быть либо 0, либо равным сумме заказов 
пришедших после даты оформления текущего заказа), а так же не имел возвратов - таких клиентов мы будем считать наиболее релевантными лидами. 
  Следующая категории
   - это те клиенты которые имеют заказы, производили оплаты и не имеют возвраты; category 1
   - это те клиенты которые имеют заказы, производили оплаты и  имеют возвраты; category 2
   - это те клиенты которые имеют заказы, не произвели оплаты; category 3
   - это те клиенты которые не делали заказов (для нас не информативны);
   - клиенты, которые сделали заказ и оплатили, но не идетифицирован источник (для нас не информативны).
  Можно бы было выделить отдельную категорию - кто произвел оплату, не сделав заказа (если приложение позволяет так делать), таких можно записать в спонсоров:)

### Реализация

In [ ]:
!pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import psycopg2
import pandas as pd

DB_HOST = 'de-edu-db.chronosavant.ru'
DB_USER = 'da11'
DB_USER_PASSWORD = 'gandalfthegrey'
DB_NAME = 'edu'

conn = psycopg2.connect(host=DB_HOST, user=DB_USER, password=DB_USER_PASSWORD, dbname=DB_NAME)

In [ ]:
/*   - это те клиенты которые имеют заказы, производили оплаты и не имеют возвраты; category 1
   - это те клиенты которые имеют заказы, производили оплаты и  имеют возвраты; category 2
   - это те клиенты которые имеют заказы, не произвели оплаты; category 3

from da.user_visits – события посещения интернет-магазина за полгода
from da.user_orders – события нажатия на кнопку "Checkout" за полгода
from da.user_payments – события оплаты за заказ за полгода
from da.user_x_channel – витрина с указанием канала привлечения для каждого идентификатор юзера
*/

In [ ]:
res=pd.read_sql_query("""
select *
from
(
select
uo.row_id, x.channel, 'category 1' as category--, count(distinct uo.user_id) as quantity_value
from da.user_orders uo
  left join da.user_x_channel x
    on uo.user_id = x.user_id
  left join da.user_payments up
    on uo.user_id = up.user_id
where channel is not Null
  and amount_refunded = 0
group by uo.row_id, x.channel,category

union all

select
uo.row_id, x.channel,'category 2' as category--, count(distinct uo.user_id) as quantity_value
from da.user_orders uo
  left join da.user_x_channel x
    on uo.user_id = x.user_id
  left join da.user_payments up
    on uo.user_id = up.user_id
where channel is not Null
  and amount_refunded > 0
group by uo.row_id, x.channel,category

union all

select
uo.row_id, x.channel,'category 3' as category--, count(distinct uo.user_id) as quantity_value
from da.user_orders uo
  left join da.user_x_channel x
    on uo.user_id = x.user_id
where channel is not Null
  and uo.user_id not in (select up.user_id from da.user_payments up)
group by uo.row_id, channel,category
) cat
order by 3
    """, conn)


In [ ]:
res.to_csv('order_acquisition.csv')
!cat order_acquisition.csv

,row_id,channel,category
0,1970.0,vk.com,category 1
1,1199.0,vk.com,category 1
2,1055.0,vk.com,category 1
3,836.0,vk.com,category 1
4,619.0,vk.com,category 1
5,543.0,vk.com,category 1
6,1403.0,vk.com,category 1
7,537.0,vk.com,category 1
8,1777.0,vk.com,category 1
9,810.0,vk.com,category 1
10,2056.0,vk.com,category 1
11,385.0,vk.com,category 1
12,2424.0,yandex direct,category 1
13,1628.0,vk.com,category 1
14,1726.0,vk.com,category 1
15,638.0,vk.com,category 1
16,383.0,vk.com,category 1
17,902.0,vk.com,category 1
18,1790.0,vk.com,category 1
19,191.0,vk.com,category 1
20,1031.0,vk.com,category 1
21,189.0,vk.com,category 1
22,2110.0,vk.com,category 1
23,1030.0,vk.com,category 1
24,3.0,yandex direct,category 1
25,2043.0,vk.com,category 1
26,2136.0,yandex direct,category 1
27,1857.0,vk.com,category 1
28,2069.0,vk.com,category 1
29,2133.0,yandex direct,category 1
30,1024.0,yandex direct,category 1
31,1492.0,vk.com,category 1
32,2331.0,vk.com,category 1
33,973.0,yandex direct,category 1
34,313